FastText is a word representation and classification tool developed by Facebook's AI Research (FAIR) lab. It extends the Word2Vec model by representing each word as a bag of character n-grams, which helps capture subword information and improve the handling of rare words.

Here's a simple implementation of FastText from scratch in Python using NumPy. This example will focus on creating word vectors using character n-grams and training a basic model to understand the concept.

In [7]:
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import normalize

class FastText:
    def __init__(self, vocab_size, embedding_dim, n_gram_size=3, learning_rate=0.01, epochs=10):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.n_gram_size = n_gram_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.word_embeddings = np.random.uniform(-0.1, 0.1, (vocab_size, embedding_dim))
        self.context_embeddings = np.random.uniform(-0.1, 0.1, (vocab_size, embedding_dim))
        self.vocab = {}
        self.rev_vocab = {}

    def build_vocab(self, sentences):
        word_count = defaultdict(int)
        for sentence in sentences:
            words = sentence.split()
            for word in words:
                word_count[word] += 1
        self.vocab = {word: idx for idx, (word, _) in enumerate(word_count.items())}
        self.rev_vocab = {idx: word for word, idx in self.vocab.items()}
    
    def get_ngrams(self, word):
        ngrams = set()
        word = '<' * (self.n_gram_size - 1) + word + '>' * (self.n_gram_size - 1)
        for i in range(len(word) - self.n_gram_size + 1):
            ngrams.add(word[i:i + self.n_gram_size])
        return ngrams

    def train(self, sentences):
        for epoch in range(self.epochs):
            loss = 0
            for sentence in sentences:
                words = sentence.split()
                for i, word in enumerate(words):
                    if word not in self.vocab:
                        continue
                    word_idx = self.vocab[word]
                    # Collect n-grams for the target word
                    target_ngrams = self.get_ngrams(word)
                    # Update context vectors
                    for j in range(max(0, i - 1), min(len(words), i + 2)):
                        if i != j and words[j] in self.vocab:
                            context_idx = self.vocab[words[j]]
                            # Update embeddings using a simple SGD approach
                            prediction = self.predict(word_idx, context_idx)
                            error = prediction - 1 if j == i + 1 else prediction
                            loss += error**2
                            self.word_embeddings[word_idx] -= self.learning_rate * error * self.context_embeddings[context_idx]
                            self.context_embeddings[context_idx] -= self.learning_rate * error * self.word_embeddings[word_idx]
            print(f'Epoch {epoch + 1}/{self.epochs}, Loss: {loss}')

    def predict(self, word_idx, context_idx):
        # Calculate the dot product of the word and context embeddings
        return np.dot(self.word_embeddings[word_idx], self.context_embeddings[context_idx])

    def get_word_vector(self, word):
        if word in self.vocab:
            return self.word_embeddings[self.vocab[word]]
        else:
            raise ValueError(f"Word '{word}' not found in vocabulary")

    def get_embedding_matrix(self):
        return normalize(self.word_embeddings, axis=1)

In [8]:
# Example usage
sentences = [
    "fast text is a library for efficient text classification",
    "word embeddings are useful for NLP tasks",
    "fasttext models can handle out-of-vocabulary words"
]

fasttext_model = FastText(vocab_size=100, embedding_dim=50)
fasttext_model.build_vocab(sentences)
fasttext_model.train(sentences)

# Get the vector for a word
vector = fasttext_model.get_word_vector("fast")
print(f"Vector for 'fast': {vector}")

Epoch 1/10, Loss: 19.119878852379344
Epoch 2/10, Loss: 18.99453025838787
Epoch 3/10, Loss: 18.870092708139556
Epoch 4/10, Loss: 18.746487508551116
Epoch 5/10, Loss: 18.623638717743134
Epoch 6/10, Loss: 18.501473031096197
Epoch 7/10, Loss: 18.3799196759705
Epoch 8/10, Loss: 18.258910314547194
Epoch 9/10, Loss: 18.13837895429284
Epoch 10/10, Loss: 18.018261865586297
Vector for 'fast': [-0.05548203 -0.03690686 -0.09314208 -0.03563283  0.06050951  0.020029
 -0.05077991  0.08787123 -0.01413946  0.0086497  -0.03687215  0.05203214
  0.01835446 -0.06844073  0.09027416 -0.02818438  0.10348338 -0.07489015
  0.07761034 -0.04408902  0.0767948  -0.06356193 -0.00514316 -0.04496691
  0.05477788  0.00410989 -0.05043605 -0.09628502  0.00932644 -0.04340611
  0.01002255  0.08901038  0.08351978  0.00404784  0.05514393  0.0440209
 -0.01103182  0.05813477  0.00603558 -0.09994054 -0.04833565 -0.03460841
 -0.05296038  0.08717721 -0.05086532 -0.10222166  0.08960338 -0.00234772
  0.05215666 -0.07103336]


### Explanation

1. **Initialization**:
   - `vocab_size`: Size of the vocabulary.
   - `embedding_dim`: Dimension of the word embeddings.
   - `n_gram_size`: Size of character n-grams.
   - `learning_rate`: Learning rate for updating embeddings.
   - `epochs`: Number of training epochs.

2. **Building Vocabulary**:
   - `build_vocab()`: Constructs the vocabulary from the sentences and creates reverse mapping.

3. **Generating N-grams**:
   - `get_ngrams()`: Generates character n-grams for a given word. The word is padded with `<` and `>` symbols to capture edge cases.

4. **Training**:
   - `train()`: Updates word and context embeddings using a simple SGD approach. The loss is computed as the squared error between the predicted and actual values.

5. **Prediction**:
   - `predict()`: Calculates the dot product between the target word and context embeddings.

6. **Getting Word Vectors**:
   - `get_word_vector()`: Retrieves the embedding for a specific word.

7. **Normalization**:
   - `get_embedding_matrix()`: Returns the normalized embedding matrix.

This code provides a simplified version of FastText. Real-world implementations are more complex, involving negative sampling, hierarchical softmax, and optimized training methods for efficiency.